#  📌 하이브리드 추천 (SVD + TF-IDF)

## 1. 한 번만 실행

### ✏️ DB 연결

In [1]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_info
dbcol_review = db.review
dbcol_feat = db.ani_feature

### ✏️ import

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### ✏️ DataFrame 가져오고 columns 수정

In [3]:
# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1, "series_id":1}))
ani_df.drop('_id', axis = 1, inplace = True)
ani_df.columns = ["ani_id", "ani_name", "series_id"]

# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["profile", "animation", "score"]]
rating_df.columns = ["user_id", "ani_id", "score"]

# 형태소 분석이 이뤄진 데이터 불러오기
feat_df = pd.DataFrame(dbcol_feat.find({},{"id":1, "feat_str":1}))

### ✏️ Matrix로 변경

In [4]:
# 사용자 - 애니 pivot table 생성
user_ani_ratings_df = rating_df.pivot(
    index='user_id',
    columns='ani_id',
    values='score'
).fillna(0)

### ✏️ id, idx, series, name, feat을 통해 다른 것을 찾는 코드들

In [5]:
ani_df_id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
ani_df_idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
ani_df_id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))
ani_df_id_to_name = dict(zip(ani_df["ani_id"], ani_df["ani_name"]))
feat_df_id_to_idx = dict(zip(feat_df["id"], feat_df.index))
feat_df_idx_to_feat = dict(zip(feat_df.index, feat_df["feat_str"]))

### ✏️ SVD를 활용한 애니메이션 30개 추천 함수

In [6]:
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 유저가 이미 본 애니 id를 뽑는다. (TF-IDF에 사용)
    user_history_list = user_history['ani_id'].values.tolist()
    
    # 유저가 이미 본 애니 series id를 뽑는다.
    user_history_series_list_all = user_history['series_id'].values.tolist()
    user_history_series_list = []
    for id in user_history_series_list_all:
        # 시리즈가 미리 추가되어 있지 않고 nan이 아니면 히스토리 시리즈 리스트에 추가
        if id not in user_history_series_list and np.isnan(id) == False:
            user_history_series_list.append(id)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list_all = list(recommendations.sort_values(userId, ascending = False)['ani_id'])

    # 추천 애니메이션 저장 리스트
    recommendation_list = []
    
    # 시리즈 저장 리스트
    series_list = user_history_series_list[:]
    
    for id in recommendation_list_all:
        
        # 선택한 애니메이션의 인덱스 가져옴
        idx = ani_df_id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = ani_df_idx_to_series[idx]
        
        # 시리즈가 미리 추가되어 있지 않다면 추천결과에 추가
        if series not in series_list:
            recommendation_list.append(id)
            # nan이 아니면 시리즈 리스트에 추가
            if np.isnan(series) == False:
                series_list.append(series)
        
        # 30개가 추천되면 종료
        if len(recommendation_list) == 30:
            return user_history_list, recommendation_list

### ✏️ TF-IDF를 활용한 애니메이션 30->14개 추천 함수

In [16]:
def recommend_ani_2(user_history_list, recommendation_list):
    tf_idf_list = list()
    user_feat = ''
    for i in user_history_list:
        idx = feat_df_id_to_idx[i]
        feat = feat_df_idx_to_feat[idx]
        user_feat += feat
        user_feat += ' '
    tf_idf_list.append(user_feat)
    for j in recommendation_list:
        idx = feat_df_id_to_idx[j]
        reco_feat = feat_df_idx_to_feat[idx]
        tf_idf_list.append(reco_feat)
    
    tf_idf = TfidfVectorizer()
    tf_idf_matrix = tf_idf.fit_transform(tf_idf_list)
    cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:15]

    id_list = list()
    for num in range(14):
        id_list.append(recommendation_list[sim_scores[num][0]-1])
    
    return id_list

## 2. 유저가 리뷰를 남길 때마다 실행?

#### 1. 리뷰를 남기면 DB와 rating_df에 데이터 추가

#### 2-1. user_ani_rating_df에 행(user_id), 열(ani_id), 값(score)를 추가
#### 2-2. 만약 리뷰를 남긴 적이 있으면 score의 값을 0점에서 평가점수로 업데이트

In [30]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = user_ani_ratings_df.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-애니에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

# U 행렬, sigma 행렬, V 전치 행렬을 반환.
U, sigma, Vt = svds(matrix_user_mean, k = 12)

# 위는 0이 아닌 값만 포함되었기에 0이 포함된 대칭행렬로 diag를 이용해 변환
sigma = np.diag(sigma)

# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 아까 평균을 빼었으니 다시 사용자 평균 rating 더한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns)

In [21]:
# input_user_id = 4523846
input_user_id = 4810880

user_history_list, recommendation_list = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df)
id_list = recommend_ani_2(user_history_list, recommendation_list)

In [24]:
print("##### 내가 본 애니!")
for result in user_history_list:
    print(ani_df_id_to_name[result])
print(" ")
print("##### 추천받은 애니!")
for result in id_list:
    print(ani_df_id_to_name[result])

##### 내가 본 애니!
귀멸의 칼날 : 환락의 거리편
호리미야
(자막) 극장판 원피스 1기 : 황금의 대해적 우난
나루토 1기
청춘 돼지는 바니걸 선배의 꿈을 꾸지 않는다
진격의 거인 The FINAL part 2
단간론파 : 희망의 학교와 절망의 고교생
나루토 질풍전 1기
주술회전 part 1
신의 탑
(자막) 시간을 달리는 소녀
글레이프니르
역시 내 청춘 러브코메디는 잘못됐다. 속 - 판권 부활
역시 내 청춘 러브코메디는 잘못됐다. 완
역시 내 청춘 러브코메디는 잘못됐다.
원펀맨
어서오세요 실력 지상주의 교실에
도쿄 리벤저스 part 1
약속의 네버랜드 1기
무직전생 : 이세계에 갔으면 최선을 다한다 1기
쓰레기의 본망
Re : 제로부터 시작하는 이세계 생활 감독판
달링 인 더 프랑키스
우리들의 리메이크
친구 게임
도메스틱한 그녀
이윽고 네가 된다
옆자리 괴물군
살육의 천사
그녀도 여친
귀엽기만 한 게 아닌 시키모리 양
소꿉친구가 절대로 지지 않는 러브 코미디
 
##### 추천받은 애니!
(자막) 나의 히어로 아카데미아 5기
괴물사변
(자막) 이 멋진 세계에 축복을! 1기
방패 용사 성공담 Season 1
전생했더니 슬라임이었던 건에 대하여 2기 1부
카구야 님은 고백받고 싶어 ~천재들의 연애 두뇌전~ 2기
나만이 없는 거리
불멸의 그대에게 part 1
목소리의 형태
어쨌든 귀여워
소드 아트 온라인 1기
4월은 너의 거짓말
암살교실 1기
월간순정 노자키 군
